In [45]:
import logging
import threading
import time

In [46]:
def thread_function(name, secondary_id):
    logging.info(f'Thread %s:%s is starting' % (name, secondary_id))
    time.sleep(2)
    logging.info(f'Thread %s:%s is ending' % (name, secondary_id))

In [47]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

In [58]:
def parent_method():
    logging.info("Main    : before creating thread")
#     x = threading.Thread(target=thread_function, args=('one', 1), daemon=True)
    x = threading.Thread(target=thread_function, args=('one', 1))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
#     x.join()
    logging.info("Main    : all done")
    
# Note that daemon setting won't work in ipython. If I had used
# __name__ == '__main__' and then invoked the script, the use of daemon setting can be demo'ed

In [60]:
parent_method()

18:35:14: Main    : before creating thread
18:35:14: Main    : before running thread
18:35:14: Thread one:1 is starting
18:35:14: Main    : wait for the thread to finish
18:35:16: Thread one:1 is ending
18:35:16: Main    : all done


In [123]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

threads = list()
for index in range(3):
    logging.info("Main    : create and start thread %d.", index)
    x = threading.Thread(target=thread_function, args=(index,))
    threads.append(x)
    x.start()

for index, thread in enumerate(threads):
    logging.info("Main    : before joining thread %d.", index)
    thread.join()
    logging.info("Main    : thread %d done", index)

10:00:32: Main    : create and start thread 0.
10:00:32: Thread 0: starting
10:00:32: Main    : create and start thread 1.
10:00:32: Thread 1: starting
10:00:32: Main    : create and start thread 2.
10:00:32: Thread 2: starting
10:00:32: Main    : before joining thread 0.
10:00:34: Thread 0: finishing
10:00:34: Main    : thread 0 done
10:00:34: Main    : before joining thread 1.
10:00:34: Thread 1: finishing
10:00:34: Main    : thread 1 done
10:00:34: Main    : before joining thread 2.
10:00:34: Thread 2: finishing
10:00:34: Main    : thread 2 done


In [124]:
# Using a thread pool executor
import concurrent.futures

# [rest of code]

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(thread_function, range(3))

10:00:38: Thread 0: starting
10:00:38: Thread 1: starting
10:00:38: Thread 2: starting
10:00:40: Thread 0: finishing
10:00:40: Thread 2: finishing
10:00:40: Thread 1: finishing


In [151]:
# class FakeDatabase:
#     def __init__(self):
#         self.value = 0

#     def update(self, name):
#         logging.info("Thread %s: starting update", name)
#         local_copy = self.value
#         local_copy += 1
#         time.sleep(0.1)
#         self.value = local_copy
#         logging.info("Thread %s: finishing update", name)

In [153]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        with self._lock:
            logging.info("Thread %s: starting update", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.info("Thread %s: finishing update", name)

In [154]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

10:44:44: Testing update. Starting value is 0.
10:44:44: Thread 0: starting update
10:44:44: Thread 0: finishing update
10:44:44: Thread 1: starting update
10:44:45: Thread 1: finishing update
10:44:45: Testing update. Ending value is 2.


In [156]:
import threading

l = threading.RLock()
print("before first acquire")
l.acquire()
print("before second acquire")
l.acquire()
print("acquired lock twice")
l.release()
l.release()

before first acquire
before second acquire
acquired lock twice


In [183]:
import random

SENTINEL = 'SENTINEL'

def producer(pipeline):
    for iteration in range(10):
        message = random.randint(1,100)
        print("Producer generating message {}".format(message))
        pipeline.set_message(message, "Producer")
        print("Producer sent message {}".format(message))
        
    pipeline.set_message(SENTINEL, "Producer")

In [184]:
def consumer(pipeline):
    message = None
    while message is not SENTINEL:
        message = pipeline.get_message("Consumer")        
        print("Consumer received message {}".format(message))

In [185]:
class Pipeline:
    def __init__(self):
        self.message = None
        self.producer_lock = threading.Lock()
        self.consumer_lock = threading.Lock()
        self.consumer_lock.acquire()
        
    def set_message(self, message, agent):
        self.producer_lock.acquire()
        self.message = message
        self.consumer_lock.release()
        
    def get_message(self, agent):
        self.consumer_lock.acquire()
        message = self.message
        self.producer_lock.release()
        return message

In [186]:
if __name__ == "__main__":
    pipeline = Pipeline()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline)
        executor.submit(consumer, pipeline)

Producer generating message 95
Producer sent message 95
Producer generating message 64
Consumer received message 95
Producer sent message 64Consumer received message 64
Producer generating message 64
Producer sent message 64
Producer generating message 73

Consumer received message 64
Producer sent message 73
Producer generating message 29
Consumer received message 73
Producer sent message 29Consumer received message 29

Producer generating message 2
Producer sent message 2
Consumer received message 2Producer generating message 84
Producer sent message 84
Producer generating message 31

Consumer received message 84
Producer sent message 31Consumer received message 31

Producer generating message 7
Producer sent message 7
Producer generating message 67
Consumer received message 7Producer sent message 67

Consumer received message 67
Consumer received message SENTINEL
